# Overview (Example: Twitter)

## Requirement clarification

- Do users post tweets and follow other people?
- Do users search tweets?
- Need to display user timeline?
- Need to push notification on new tweets?
- Need to display trending topics?
- Do tweets contain photos and videos?
- Are we discussing backend only or frontend too?

## System estimation

- Numer of tweets? Number of users? 
- Size of storage?
- Network bandwidth?

## System interface

- What APIs are expected?

## Data model

- User (UserID, Name, Email, CreationDate, LastLogin, etc)
- Tweet (TweetID, Content, TimeStamp, etc)
- UserFollow (UserID1, UserID2)

## High-level design

- Multiple app servers to do read/write with LoadBalancer in front.
- Efficient DB to store all tweets and support large number of reads.
- Distributed file storage to store photos and videos.

## Detailed design

- How to partition data and distribute it to multiple DBs.
- How to handle "hot" users.
- At which layer should we introduce cache. 

## Identify bottleneck

- Is there a single point of failure?
- Do we have replicas of data?
- Do we have availability set of app servers?
- Do we monitor servers and get alerts?

# Design TinyURL (URL shortening)

- Short link. This is used to save a lot of space. This URL must not be guessable.
- When users clock short link, they are redirected to the original URL.

## Capacitiy Estimate

Example: traffic
- 500M URL shortening per month.
- 100:1 read:write ratio.
- Numer of redirections per month: 100 * 500M = 50B
- Queries per second: 500M / (30 days * 24 hrs * 3600 seconds) = 200 URLs/s
- URL redirections per second: 100 * 200 URLs/s = 20k URLs/s 

Example: storage
- Assume we store URL shortening request for 5 yrs.
- Total number of objects to store: 500M * 5 yrs * 12 months = 30 billion
- Assume each object is around 500 bytes.
- Total storage: 30 billion * 500 bytes = 15TB

Example: Bandwidth
- Write (Queries per second is 200 URLs): 200 * 500 bytes = 100 kB/s
- Read (URL redirections per second is 20k URLs): 20k * 500 bytes = 10 MB/s

Example: Memory
- Assume 20% of URLs generate 80% of traffic. (Hot URLs)
- Request per day: 20k URLs/s * 3600 seconds * 24 hrs = 1.7 billion
- To cache 20% of these requests: 0.2 * 1.7 billion * 500 bytes = 170 GB
    - There will be duplicate requests of the same URLs, so the actual required memory will be less.
    
## System API

- createURL(api_dev_key, original_url)
    - api_dev_key: API developer key of registered account.
    - original_url: URL to be shortened.
- deleteURL(api_dev_key, url_key)
    - url_key: shortened URL.
- To prevent user abuse, limit "api_dev_key" to certain number of creations and redirections per time period.     
   
## Database

- Billions of records.
- Each object is small. (500k)
- Read heavy.
- Since there is no relationship between records, No SQL should be chosen.
- Schema "URL": Hash(varchar 16, PK), OriginalURL(varchar), CreationDate(datetime), ExpirationDate(datetime), UserID(int) 
- Schema "User": UserID(int), Name(varchar), Email(varchar), CreationDate(datetime), LastLogin(datetime)

## Encoding algorithm

- Want to generate characters at the end of the URL.
- Assume base64 encoding.
    - 6 letters 64^6 = 68.7 billion possible strings.
    - 8 letters 64^8 = 281 trillion possible strings.
- If multiple users enter the same URL, they can get the same shortened URL, which is not good.
- What if randomly generate 6 letter strings beforehand and store them into DB?
    - If a key is used, then it should be marked as used.
    - What is two or more servers are trying the use the same key?
        - One table for keys not in used, one table for keys in use.
        - Load some keys in memory to give to the servers and at the same time move them to "Used" table.
     - DB size: 6 (characters per key) * 68.7 billion (unique keys) = 412GB

## Data partitioning and replication

- Range based partitioning
    - Partiion based on the first letter of URL hash keys.
    - This could lead to unbalanced partitioning.
- Hash based partitioning
    - Use hash function to map "key" to a number between [1,255]

## Cache

- Cache URLs that are frequently accessed.

## Load balancer

- Between Client and App servers.
- Between App servers and DB servers.
- Between App servers and cache servers.

## DB clean up

- Lightweight cleanup service that removes expired links from cache and storage (Also put the key back into key DB to be reused)